In [ ]:
IDs = list()                #.I
titles = list()             #.T
abstracts = list()          #.W
pub_dates = list()          #.B
authors = list()            #.A
info = list()               #.N
refrences = list()          #.X

In [23]:
#read CISI.ALL file and fill lists

with open("CISI.ALL", encoding = "latin-1") as all_files:
    for line in all_files.readlines():
        if line.startswith(".I"):
            t = 0
            a = 0
            w = 0
            x = 0
            n = 0
            b = 0
            id = line[3]
            IDs.append(id)
        elif line.startswith(".T"):
            t = 1
            a = 0
            w = 0
            x = 0
            n = 0
            b = 0
        elif line.startswith(".W"):
            t = 0
            a = 0
            w = 1
            x = 0
            n = 0
            b = 0
        elif line.startswith(".B"):
            t = 0
            a = 0
            w = 0
            x = 0
            n = 0
            b = 1
        elif line.startswith(".N"):
            t = 0
            a = 0
            w = 0
            x = 0
            n = 1
            b = 0
        elif line.startswith(".X"):
            t = 0
            a = 0
            w = 0
            x = 1
            n = 0
            b = 0
        elif line.startswith(".A"):
            t = 0
            a = 1
            w = 0
            x = 0
            n = 0
            b = 0
        elif t == 1:
            # title
            titles.append(line)
            t = 0
        elif a == 1:
            # author
            authors.append(line)
        elif b == 1:
            # publication date
            pub_dates.append(line)
        elif w == 1:
            # abstract
            abstracts.append(line)
        elif x == 1:
            # references
            refrences.append(line)
        elif n == 1:
            # info
            info.append(line)
        else:
            continue
